# All flights

In [8]:
from opensky_api import OpenSkyApi
import pandas as pd

# Initialize the API (can also be authenticated) 
# dont be dumb if you have the keys

api = OpenSkyApi('JetRider', 'wiwnib-bizsez-5daTjo')

# Fetch the current states of all aircraft
states = api.get_states()

# Extract data from each StateVector and store in a list
data = []
for s in states.states:
    data.append({
        "ICAO24": s.icao24,
        "Callsign": s.callsign,
        "Origin Country": s.origin_country,
        "Latitude": s.latitude,
        "Longitude": s.longitude,
        "Baro Altitude": s.baro_altitude,
        "Geo Altitude": s.geo_altitude,
        "Velocity": s.velocity,
        "True Track": s.true_track,
        "Vertical Rate": s.vertical_rate,
        "On Ground": s.on_ground,
        "Category": s.category,
        "Last Contact": s.last_contact,
        "Position Source": s.position_source,
        "Sensors": s.sensors,
        "SPI": s.spi,
        "Squawk": s.squawk,
        "Time Position": s.time_position
    })

# Convert list to DataFrame
df = pd.DataFrame(data)
print(f'Flights Found: {len(df)}')
# Display the first few rows
print(df.head())

# show flights on map using folium
import folium
from folium.plugins import MarkerCluster

# Create a map
m_1 = folium.Map(location=[39.8283, -98.5795], tiles='cartodbpositron', zoom_start=4)

# Add points to the map
mc = MarkerCluster()
for idx, row in df.iterrows():
    if not pd.isnull(row['Longitude']) and not pd.isnull(row['Latitude']):
        mc.add_child(folium.Marker([row['Latitude'], row['Longitude']]))
m_1.add_child(mc)

# Display the map
m_1


Flights Found: 7576
   ICAO24  Callsign Origin Country  Latitude  Longitude  Baro Altitude  \
0  e49406  GLO1783          Brazil  -15.8860   -48.0217        1463.04   
1  c07d0b  CGVJE            Canada   45.4860   -73.2036         518.16   
2  a09281  N136LM    United States   30.2711   -87.6222        1310.64   
3  aa8c39  UAL1493   United States   39.9856   -83.9688       11582.40   
4  e8027e  LPE2675           Chile  -10.9505   -77.6418        5798.82   

   Geo Altitude  Velocity  True Track  Vertical Rate  On Ground  Category  \
0       1539.24     79.87       86.68          -3.58      False         0   
1        525.78     55.58      239.40          -1.95      False         0   
2       1386.84     54.37      263.48          -0.98      False         2   
3      12001.50    216.91      287.10           0.00      False         1   
4           NaN    219.73      154.03         -13.00      False         1   

   Last Contact  Position Source Sensors    SPI Squawk  Time Position  


In [7]:
len(df)

7624

In [2]:
from opensky_api import OpenSkyApi
import folium

# dont screw me here pls
# Replace with your OpenSky credentials
api = OpenSkyApi('JetRider', 'wiwnib-bizsez-5daTjo') 



# Provided tail numbers and their corresponding Mode S/ICAO24 codes
# TAIL_NUMBER_TO_ICAO24 = {
#     "N874SA": "ac06C3",
#     "N853SA": "abb40e",
#     "N828SA": "ab5024",
#     "N821SA": "ab3623"
# }


# List of icao24 codes for the aircrafts you are interested in
icao24_list = ['ac06C3']

# Get the states of the aircrafts in the icao24 list
states = api.get_states(icao24=icao24_list)

# Get the flight data by aircraft for the time interval mentioned
flights_by_aircraft = api.get_flights_by_aircraft(icao24_list, 1517227200, 1517230800)
print(flights_by_aircraft)

# Check if we have any states
if states and states.states:
    for s in states.states:
        # Set the current state
        current_state = s
        print('Current Position')
        print(current_state)

    # Get the track of a specific aircraft (I noticed you used a different icao24 here, adjust as necessary)
    track = api.get_track_by_aircraft("a0fe7a")
    print('Track')
    print(track)

    # Get the path data from the track
    path_data = track.path

    # Extract latitudes and longitudes from the path data
    latitudes = [point[1] for point in path_data]
    longitudes = [point[2] for point in path_data]

    # Calculate the average latitude and longitude to center the map
    avg_latitude = sum(latitudes) / len(latitudes)
    avg_longitude = sum(longitudes) / len(longitudes)

    # Create a map centered around the average latitude and longitude
    m = folium.Map(location=[avg_latitude, avg_longitude], zoom_start=8)

    # Plot the flight path on the map
    flight_path = list(zip(latitudes, longitudes))
    folium.PolyLine(flight_path, color="blue", weight=2.5, opacity=1).add_to(m)

    # Add a marker for the current position of the aircraft
    current_position = {
        'latitude': current_state.latitude,
        'longitude': current_state.longitude
    }
    folium.Marker([current_position['latitude'], current_position['longitude']], tooltip="Current Position").add_to(m)

    # Display the map
    m
else:
    print("No states found for the given icao24 list.")



None
No states found for the given icao24 list.
